# #100 User Vehicles
<i>Create Discrete Load attributes representing vehicle axle arrangements defined in a spreadsheet</i>
***

In [1]:
file_path = "DataFiles/User Vehicles.xlsx" # Path to the Excel file containing vehicle data
vehicles_to_generate = [] # Optional list of vehicles to generate, leave empty to generate all vehicles

In [2]:
# Read definitions file
import pandas as pd
df = pd.read_excel(file_path, sheet_name="Vehicles", usecols=range(0,5))

In [3]:
# Filter the list of vehicles if required
if len(vehicles_to_generate) > 0:
    df = df[df.Name.isin(vehicles_to_generate)] 

In [4]:
import sys; sys.path.append('../') # Reference modules in parent directory
from shared.LPI_21_1 import *
lusas = get_lusas_modeller()

if not lusas.existsDatabase():
    raise Exception("This script will add loadcases to an an existing model, please open a model and run the script again.")

# Reference the current database for convenience
db = lusas.database()
# Get the current database units
model_units = db.getModelUnits()

In [5]:
# Loop through each of the vehicle definitions in the sheet, each one is defined on a single row
for i, row in df.iterrows():
    # Name of the vehicle attribute to be created - Will overwrite existing attributes with the same name
    name = row.Name
    # Read the axle spacing, axle load and axle width
    spacings = str(row.Spacings).split("|")
    loads    = str(row.Loads).split("|")
    widths   = str(row.Widths).split("|")
    # Check that the number of each are consistent
    if len(spacings) == len(loads) == len(widths):
        
        try:
            # Get the unit set for the vehicle definitions
            vehicle_units = lusas.getUnitSet(row.Units)
            # Create the discrete point load attribute with loads acting in the Z direction
            vehicle_attr = db.createLoadingDiscretePoint(name)
            vehicle_attr.setDiscrete("Z", [0,0,-1], len(spacings), 2)
            
            # Longitudinal position of each axle
            x = 0
            for i in range(0, len(spacings)):
                # Get the details of the current axle
                s = float(spacings[i])
                l = float(loads[i])
                w = float(widths[i])
                # Convert values to the model unit set
                s = vehicle_units.convertToUnitSet(model_units, s, 0,0,1,0,0,0)
                l = vehicle_units.convertToUnitSet(model_units, l, 0,1,0,0,0,0)
                w = vehicle_units.convertToUnitSet(model_units, w, 0,0,1,0,0,0)
                # Increment the axle position by the spacing
                x-=s
                # Add wheel loads for the axle
                vehicle_attr.addRow(x,  w/2, 0, -abs(l/2))
                vehicle_attr.addRow(x, -w/2, 0, -abs(l/2))

        except Exception as e:
            print(f"Failed creating vehicle {name}")
            print(e)
    else:
        print(f"The number of values in {name} is inconsistent")
